# UTSA CS 3793: Assignment-2

**Garcia - Jeremiah - (lnm248)**






## Learning Objectives



*   Game Playing
*   Chess - Board Setup & Rules
*   Adversarial Search
*   AI - Random vs MinMax



## Description

This assignment is focused on **game playing** and creating a proper **AI for chess**.
In the following sections, you will complete a series of tasks to create a chess game board, rules for each chess piece, a Random AI and a MinMax AI that plays a game of chess for both players (white and black).

The base structure of the code is provided. You are supposed to write code for each of the functions. Comments are provided on what should be done. You **CANNOT** use a complete chess library and change the base code structure completely. However, you **CAN** change the code layout and name/format of the functions.


#Chess Board Setup & Rules

In this section, you will write code to import the necessary libraries and create:

1.   **ChessBoard** - This part will contain code to initialize the board, draw the board, get the board state and move piece.
2.   **ChessRules** - This part will contain code for the chess rules for each piece.



##Import Libraries

The code here will contain only **import** statements. A base list of the required libraries are already imported. You will most likely not need any other libraries, but if needed, add the import statements here. As mentioned before, you can not use any premade chess libraries.

In [1]:
# IMPORTANT NOTE: WHENEVER TESTING CODE OPT FOR "RUN ALL" OVER RUNNING INDIVIDUAL CELLS AS DEFINITIONS ARE MADE DEEPER IN THE CODE
import random
import math
import operator
import operator
from itertools import product

##ChessBoard

(10 points)

Fill the code in the code structure provided below for the ChessBoard. The main use of this code block write functions to initialize the board, draw the board, get the board state and move piece. You can add any other functions if needed.


In [2]:
class Board:
    whites = []
    blacks = []
    def __init__(self):
        self.board = []
        self.game_mode = 0
        # for 4-ply extra credit
        self.depth = 4
        self.ai = True

    def initialize_board(self):
        for i in range(8):
            self.board.append(['empty-block' for _ in range(8)])

    # initialize and return a chess board - create a 2D 8x8 array that has the value for each cell
    # USE the following characters for the chess pieces - lower-case for BLACK and upper-case for WHITE
    # . for empty board cell
    # p/P for pawn
    # r/R for rook
    # t/T for knight
    # b/B for bishop
    # q/Q for queen
    # k/K for king
    def ChessBoardSetup(self):
        self.board.clear()
        self.whites.clear()
        self.blacks.clear()
        self.initialize_board()
        self.whiteKing = King('white', 0, 4, 'K')
        self.blackKing = King('black', 7, 4, 'k')
        for j in range(8):
            self[1][j] = Pawn('white', 1, j, 'P')
            self[6][j] = Pawn('black', 6, j, 'p')
        self[0][0] = Rook('white', 0, 0, 'R')
        self[0][7] = Rook('white', 0, 7, 'R')
        self[0][1] = Knight('white', 0, 1, 'T')
        self[0][6] = Knight('white', 0, 6, 'T')
        self[0][2] = Bishop('white', 0, 2, 'B')
        self[0][5] = Bishop('white', 0, 5, 'B')
        self[0][3] = Queen('white', 0, 3, 'Q')
        self[0][4] = self.whiteKing
        self[7][0] = Rook('black', 7, 0, 'r')
        self[7][7] = Rook('black', 7, 7, 'r')
        self[7][1] = Knight('black', 7, 1, 't')
        self[7][6] = Knight('black', 7, 6, 't')
        self[7][2] = Bishop('black', 7, 2, 'b')
        self[7][5] = Bishop('black', 7, 5, 'b')
        self[7][3] = Queen('black', 7, 3, 'q')
        self[7][4] = self.blackKing

        self.SavePieces()

    def SavePieces(self):
        for i in range(8):
            for j in range(8):
                if isinstance(self[i][j], ChessPiece):
                    if self[i][j].color == 'white':
                        self.whites.append(self[i][j])
                    else:
                        self.blacks.append(self[i][j])

    def MovePiece(self, piece, x, y, keep_history=False):
        old_x = piece.x
        old_y = piece.y
        if keep_history:
            self.board[old_x][old_y].SetLastEaten(self.board[x][y])
        else:
            if isinstance(self.board[x][y], ChessPiece):
                if self.board[x][y].color == 'white':
                    print("White lost a piece!")
                    self.whites.remove(self.board[x][y])
                else:
                    print("Black lost a piece!")
                    self.blacks.remove(self.board[x][y])
        self.board[x][y] = self.board[old_x][old_y]
        self.board[old_x][old_y] = 'empty-block'
        self.board[x][y].SetPosition(x, y, keep_history)

    def UndoMove(self, piece):
        x = piece.x
        y = piece.y
        self.board[x][y].SetOldPosition()
        old_x = piece.x
        old_y = piece.y
        self.board[old_x][old_y] = self.board[x][y]
        self.board[x][y] = piece.GetLastEaten()

    def __getitem__(self, item):
        return self.board[item]

    def has_opponent(self, piece, x, y):
        if not self.IsMoveLegal(x, y):
            return False
        if isinstance(self.board[x][y], ChessPiece):
            return piece.color != self[x][y].color
        return False

    def has_friend(self, piece, x, y):
        if not self.IsMoveLegal(x, y):
            return False
        if isinstance(self[x][y], ChessPiece):
            return piece.color == self[x][y].color
        return False

    @staticmethod
    def IsMoveLegal(x, y):
        return 0 <= x < 8 and 0 <= y < 8

    def IsClearPath(self, x, y):
        if not self.IsMoveLegal(x, y):
            return False
        return not isinstance(self[x][y], ChessPiece)

    def IsInCheck(self, color, move=None):
        if color == 'white':
            enemies = self.blacks
            king = self.whiteKing
        else:
            enemies = self.whites
            king = self.blackKing
        threats = []
        for enemy in enemies:
            moves = enemy.GetListOfLegalMoves(self)
            if (king.x, king.y) in moves:
                threats.append(enemy)
        if move and len(threats) == 1 and threats[0].x == move[0] and threats[0].y == move[1]:
            return False
        return True if len(threats) > 0 else False

    def IsCheckmateDraw(self):
        terminal1 = self.WhiteWon()
        terminal2 = self.BlackWon()
        terminal3 = self.draw()
        return terminal1 or terminal2 or terminal3

    def draw(self):
        if not self.IsInCheck('white') and not self.HasMoves('white'):
            return True
        if not self.IsInCheck('black') and not self.HasMoves('black'):
            return True
        if self.InsufficientMaterial():
            return True
        return False

    def WhiteWon(self):
        if self.IsInCheck('black') and not self.HasMoves('black'):
            return True
        return False

    def BlackWon(self):
        if self.IsInCheck('white') and not self.HasMoves('white'):
            return True
        return False

    def HasMoves(self, color):
        total_moves = 0
        for i in range(8):
            for j in range(8):
                if isinstance(self[i][j], ChessPiece) and self[i][j].color == color:
                    piece = self[i][j]
                    total_moves += len(piece.FilterMoves(piece.GetListOfLegalMoves(self), self))
                    if total_moves > 0:
                        return True
        return False

    def InsufficientMaterial(self):
        total_white_knights = 0
        total_black_knights = 0
        total_white_bishops = 0
        total_black_bishops = 0
        total_other_white_pieces = 0
        total_other_black_pieces = 0

        for piece in self.whites:
            if piece.type == 'Knight':
                total_white_knights += 1
            elif piece.type == 'Bishop':
                total_white_bishops += 1
            elif piece.type != 'King':
                total_other_white_pieces += 1

        for piece in self.blacks:
            if piece.type == 'Knight':
                total_black_knights += 1
            elif piece.type == 'Bishop':
                total_black_bishops += 1
            elif piece.type != 'King':
                total_other_black_pieces += 1

        weak_white_pieces = total_white_bishops + total_white_knights
        weak_black_pieces = total_black_bishops + total_black_knights

        if self.whiteKing and self.blackKing:
            if weak_white_pieces + total_other_white_pieces + weak_black_pieces + total_other_black_pieces == 0:
                return True
            if weak_white_pieces + total_other_white_pieces == 0:
                if weak_black_pieces == 1:
                    return True
            if weak_black_pieces + total_other_black_pieces == 0:
                if weak_white_pieces == 1:
                    return True
            if len(self.whites) == 1 and len(self.blacks) == 16 or len(self.blacks) == 1 and len(self.whites) == 16:
                return True
            if total_white_knights == weak_white_pieces + total_other_white_pieces and len(self.blacks) == 1:
                return True
            if total_black_knights == weak_black_pieces + total_other_black_pieces and len(self.whites) == 1:
                return True
            if weak_white_pieces == weak_black_pieces == 1 and total_other_white_pieces == total_other_black_pieces == 0:
                return True

    def Evaluate(self):
        white_points = 0
        black_points = 0
        for i in range(8):
            for j in range(8):
                if isinstance(self[i][j], ChessPiece):
                    piece = self[i][j]
                    if piece.color == 'white':
                        white_points += piece.GetScore()
                    else:
                        black_points += piece.GetScore()
        if self.game_mode == 0:
            return black_points - white_points
        return white_points - black_points

    def GetKing(self, piece):
        if piece.color == 'white':
            return self.whiteKing
        return self.blackKing

    def GetKing(self, piece):
        if piece.color == 'white':
            return self.whiteKing
        return self.blackKing

##ChessRules

(50 points)

Fill the code in the code structure provided below for ChessRules. The main use of the code block is to write functions to design the rules for movement of each piece on the board. This block will also contain the function to check if the current player is in check, check-mate. You can also have functions that can return the current player's pieces that have legal moves in the current board state.

Following are some **suggested** functions with the pseudocode provided. You can create/remove functions as needed.


In [3]:
class ChessPiece:
    eaten_pieces_history = []
    has_moved_history = []
    position_history = []

    def __init__(self, color, x, y, letter):
        self.moved = False
        self.color = color
        self.x = x
        self.y = y
        self.type = self.__class__.__name__
        self.letter = letter

    def FilterMoves(self, moves, board):
        final_moves = moves[:]
        for move in moves:
            board.MovePiece(self, move[0], move[1], keep_history=True)
            if board.IsInCheck(self.color, move):
                final_moves.remove(move)
            board.UndoMove(self)
        return final_moves

    def GetListOfLegalMoves(self, board):
        pass

    def GetLastEaten(self):
        return self.eaten_pieces_history.pop()

    def SetLastEaten(self, piece):
        self.eaten_pieces_history.append(piece)

    def SetPosition(self, x, y, keep_history):
        if keep_history:
            self.position_history.append(self.x)
            self.position_history.append(self.y)
            self.has_moved_history.append(self.moved)
        self.x = x
        self.y = y
        self.moved = True

    def SetOldPosition(self):
        position_y = self.position_history.pop()
        position_x = self.position_history.pop()
        self.y = position_y
        self.x = position_x
        self.moved = self.has_moved_history.pop()

    def GetScore(self):
        return 0


class Pawn(ChessPiece):

    def GetListOfLegalMoves(self, board):
        moves = []
        if board.game_mode == 0 and self.color == 'white' or board.game_mode == 1 and self.color == 'black':
            direction = 1
        else:
            direction = -1
        x = self.x + direction
        if board.IsClearPath(x, self.y):
            moves.append((x, self.y))
            if self.moved is False and board.IsClearPath(x + direction, self.y):
                moves.append((x + direction, self.y))
        if board.IsMoveLegal(x, self.y - 1):
            if board.has_opponent(self, x, self.y - 1):
                moves.append((x, self.y - 1))
        if board.IsMoveLegal(self.x + direction, self.y + 1):
            if board.has_opponent(self, x, self.y + 1):
                moves.append((x, self.y + 1))
        return moves

    def GetScore(self):
        return 10


class Knight(ChessPiece):

    def GetListOfLegalMoves(self, board):
        moves = []
        add = operator.add
        sub = operator.sub
        op_list = [(add, sub), (sub, add), (add, add), (sub, sub)]
        nums = [(1, 2), (2, 1)]
        combinations = list(product(op_list, nums))
        for comb in combinations:
            x = comb[0][0](self.x, comb[1][0])
            y = comb[0][1](self.y, comb[1][1])
            if board.IsClearPath(x, y) or board.has_opponent(self, x, y):
                moves.append((x, y))
        return moves

    def GetScore(self):
        return 20


class Bishop(ChessPiece):

    def GetListOfLegalMoves(self, board):
        moves = []
        add = operator.add
        sub = operator.sub
        operators = [(add, add), (add, sub), (sub, add), (sub, sub)]
        for ops in operators:
            for i in range(1, 9):
                x = ops[0](self.x, i)
                y = ops[1](self.y, i)
                if not board.IsMoveLegal(x, y) or board.has_friend(self, x, y):
                    break
                if board.IsClearPath(x, y):
                    moves.append((x, y))
                if board.has_opponent(self, x, y):
                    moves.append((x, y))
                    break
        return moves

    def GetScore(self):
        return 30


class Rook(ChessPiece):

    def GetListOfLegalMoves(self, board):
        moves = []
        moves += self.GetVerticalMoves(board)
        moves += self.GetHorizontalMoves(board)
        return moves

    def GetVerticalMoves(self, board):
        moves = []
        for op in [operator.add, operator.sub]:
            for i in range(1, 9):
                x = op(self.x, i)
                if not board.IsMoveLegal(x, self.y) or board.has_friend(self, x, self.y):
                    break
                if board.IsClearPath(x, self.y):
                    moves.append((x, self.y))
                if board.has_opponent(self, x, self.y):
                    moves.append((x, self.y))
                    break
        return moves

    def GetHorizontalMoves(self, board):
        moves = []
        for op in [operator.add, operator.sub]:
            for i in range(1, 9):
                y = op(self.y, i)
                if not board.IsMoveLegal(self.x, y) or board.has_friend(self, self.x, y):
                    break
                if board.IsClearPath(self.x, y):
                    moves.append((self.x, y))
                if board.has_opponent(self, self.x, y):
                    moves.append((self.x, y))
                    break
        return moves

    def GetScore(self):
        return 30


class Queen(ChessPiece):

    def GetListOfLegalMoves(self, board):
        moves = []
        rook = Rook(self.color, self.x, self.y, self.letter)
        bishop = Bishop(self.color, self.x, self.y, self.letter)
        rook_moves = rook.GetListOfLegalMoves(board)
        bishop_moves = bishop.GetListOfLegalMoves(board)
        if rook_moves:
            moves.extend(rook_moves)
        if bishop_moves:
            moves.extend(bishop_moves)
        return moves

    def GetScore(self):
        return 240


class King(ChessPiece):

    def GetListOfLegalMoves(self, board):
        moves = []
        moves += self.GetHorizontalMoves(board)
        moves += self.GetVerticalMoves(board)
        return moves

    def GetVerticalMoves(self, board):
        moves = []
        for op in [operator.add, operator.sub]:
            x = op(self.x, 1)
            if board.IsClearPath(x, self.y) or board.has_opponent(self, x, self.y):
                moves.append((x, self.y))
            if board.IsClearPath(x, self.y + 1) or board.has_opponent(self, x, self.y + 1):
                moves.append((x, self.y+1))
            if board.IsClearPath(x, self.y - 1) or board.has_opponent(self, x, self.y - 1):
                moves.append((x, self.y - 1))
        return moves

    def GetHorizontalMoves(self, board):
        moves = []
        for op in [operator.add, operator.sub]:
            y = op(self.y, 1)
            if board.IsClearPath(self.x, y) or board.has_opponent(self, self.x, y):
                moves.append((self.x, y))
        return moves

    def GetScore(self):
        return 1000

#Artificial Intelligence

In this section, you will write code for the Artificial Intelligence (AI) that will play a game of chess. You will write 2 types of AI:

1.   **RandomAI** - This part will contain code for moving a chess piece randomly.
2.   **MinMaxAI** - This part will contain code for moving a chess piece using the MinMax strategy discussed in the lecture.


##RandomAI

(10 points)

Complete the function below that will perform a random move for the given player. The function will return the move (from-piece and to-piece). You will most likely not need to write any other function, but you can, if needed.


In [4]:
def GetRandomMove(board):
    pieces = []
    moves = []
    for i in range(8):
        for j in range(8):
            if isinstance(board[i][j], ChessPiece) and board[i][j].color != 'black':
                pieces.append(board[i][j])
    for piece in pieces[:]:
        piece_moves = piece.FilterMoves(piece.GetListOfLegalMoves(board), board)
        if len(piece_moves) == 0:
            pieces.remove(piece)
        else:
            moves.append(piece_moves)
    if len(pieces) == 0:
        return
    piece = random.choice(pieces)
    move = random.choice(moves[pieces.index(piece)])
    if isinstance(piece, ChessPiece) and len(move) > 0:
        board.MovePiece(piece, move[0], move[1])
        print(piece.color.capitalize() + " " + piece.type + " to " + "(" + str(piece.y) + "," + str(piece.x) + ")")
        print()

##MinMaxAI

(50 points)

Complete the functions below that will perform a move for the given player using the MinMax AI strategy. One function will Evaluate the board if a move is performed - give score for each of piece and calculate the score for the entire chess board. In the second function you will write actual code for the MinMax strategy and return the move (from-piece and to-piece). To get the allocated points, searching should be **2-ply (one Max and one Min)**. You will most likely not need to write any other function, but you can, if needed.

## Extra Credit

*   **(5 points)** Modify the above MinMax strategy to be **4-ply (one Max, one Min, one Max, one Min)**.
*   **(10 points)** Perform **alpha-beta pruning** for the MinMax strategy.

In [5]:
# extra credit is attempted for the following implementation
def MinMax(board, depth, alpha, beta, max_player, save_move, data):

    if depth == 0 or board.IsCheckmateDraw():
        data[1] = board.Evaluate()
        return data

    if max_player:
        max_eval = -math.inf
        for i in range(8):
            for j in range(8):
                if isinstance(board[i][j], ChessPiece) and board[i][j].color != 'white':
                    piece = board[i][j]
                    moves = piece.FilterMoves(piece.GetListOfLegalMoves(board), board)
                    for move in moves:
                        board.MovePiece(piece, move[0], move[1], keep_history=True)
                        evaluation = MinMax(board, depth - 1, alpha, beta, False, False, data)[1]
                        if save_move:
                            if evaluation >= max_eval:
                                if evaluation > data[1]:
                                    data.clear()
                                    data[1] = evaluation
                                    data[0] = [piece, move, evaluation]
                                elif evaluation == data[1]:
                                    data[0].append([piece, move, evaluation])
                        board.UndoMove(piece)
                        max_eval = max(max_eval, evaluation)
                        alpha = max(alpha, evaluation)
                        if beta <= alpha:
                            break
        return data
    else:
        min_eval = math.inf
        for i in range(8):
            for j in range(8):
                if isinstance(board[i][j], ChessPiece) and board[i][j].color == 'white':
                    piece = board[i][j]
                    moves = piece.GetListOfLegalMoves(board)
                    for move in moves:
                        board.MovePiece(piece, move[0], move[1], keep_history=True)
                        evaluation = MinMax(board, depth - 1, alpha, beta, True, False, data)[1]
                        board.UndoMove(piece)
                        min_eval = min(min_eval, evaluation)
                        beta = min(beta, evaluation)
                        if beta <= alpha:
                            break
        return data


def GetMinMaxMove(board):
    moves = MinMax(board, board.depth, -math.inf, math.inf, True, True, [[], 0])
    if len(moves[0]) == 0:
        return False
    best_score = max(moves[0], key=lambda x: x[2])[2]
    piece_and_move = random.choice([move for move in moves[0] if move[2] == best_score])
    piece = piece_and_move[0]
    move = piece_and_move[1]
    if isinstance(piece, ChessPiece) and len(move) > 0 and isinstance(move, tuple):
        board.MovePiece(piece, move[0], move[1])
        print(piece.color.capitalize() + " " + piece.type + " to " + "(" + str(piece.y) + "," + str(piece.x) + ")")
        print()
    return True

#Game Setup & Main Loop

(05 points)

Write code below to have a game-play between two AIs - Random vs MinMax. For each iteration, draw the board before and after a turn.

In [6]:
# white: RandomAI
# black: MinMaxAI

def DrawBoard(board):

    piece_to_char = {
        'Pawn': 'P', 'Knight': 'T', 'Bishop': 'B', 'Rook': 'R',
        'Queen': 'Q', 'King': 'K', 'empty-block': '.'
    }

    reversed_rows = []

    for row in board:
        reversed_row = []
        for piece in row:
            char = piece_to_char.get(piece.__class__.__name__, '.')
            if piece == 'empty-block':
                char = '.'
            else:
                if getattr(piece, 'color', '') == 'black':
                    char = char.lower()
            reversed_row.append(char)
        reversed_rows.append(' '.join(reversed_row))

    for reversed_row in reversed(reversed_rows):
        print(reversed_row)


def start(board):
    running = True
    turn = 'white'
    while running:

        if turn == 'white':
            print()
            print("White's (Random AI) turn")
            GetRandomMove(board)
            DrawBoard(board)
            print()
        else:
            print()
            print("Black's (MinMax AI) turn")
            GetMinMaxMove(board)
            DrawBoard(board)
            print()

        if board.WhiteWon():
            running = False
            print("Game over: White wins")
        elif board.BlackWon():
            running = False
            print("Game over: Black wins")
        elif board.draw():
            running = False
            print("Game over: Draw")
        else:
            turn = 'black' if turn == 'white' else 'white'

if __name__ == '__main__':
    keep_playing = True
    board = Board()

    board.ChessBoardSetup()
    print("CS3793 Assignment 2 - Programmed by Jeremiah Garcia (lnm248)")
    print("Game begin")
    print()
    DrawBoard(board)
    keep_playing = start(board)

CS3793 Assignment 2 - Programmed by Jeremiah Garcia (lnm248)
Game begin

r t b q k b t r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R T B Q K B T R

White's (Random AI) turn
White Pawn to (4,3)

r t b q k b t r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R T B Q K B T R


Black's (MinMax AI) turn
Black Pawn to (5,4)

r t b q k b t r
p p p p p . p p
. . . . . . . .
. . . . . p . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R T B Q K B T R


White's (Random AI) turn
Black lost a piece!
White Pawn to (5,4)

r t b q k b t r
p p p p p . p p
. . . . . . . .
. . . . . P . .
. . . . . . . .
. . . . . . . .
P P P P . P P P
R T B Q K B T R


Black's (MinMax AI) turn
Black Pawn to (3,5)

r t b q k b t r
p p p . p . p p
. . . p . . . .
. . . . . P . .
. . . . . . . .
. . . . . . . .
P P P P . P P P
R T B Q K B T R


White's (Random AI) turn
White King to (4,1)

r t b q k b t r
p p p . p . 

#Submission Instructions

1.   Complete all tasks above - **File MUST contain the output for ALL cells**
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard



##Rubric

*   (10 points) Chess Board Setup
*   (50 points) Chess Rules Setup
*   (10 points) Random AI
*   (50 points) MinMax AI (2-ply)
*   (05 points) Game Main Loop - Random vs MinMax
*   (15 points) Extra Credit - 4-ply MinMax + alpha-beta pruning



